# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 14 2022 9:50AM,246383,10,PRF-36984,Bio-PRF,Released
1,Sep 14 2022 9:50AM,246383,10,PRF-36990,Bio-PRF,Released
2,Sep 14 2022 9:50AM,246383,10,PRF-36998,Bio-PRF,Released
3,Sep 14 2022 9:50AM,246383,10,PRF-37004,Bio-PRF,Released
4,Sep 14 2022 9:39AM,246382,12,HH-36900,Hush Hush,Released
5,Sep 14 2022 9:39AM,246382,12,HH-36901,Hush Hush,Released
6,Sep 14 2022 9:39AM,246382,12,HH-36902,Hush Hush,Released
7,Sep 14 2022 9:39AM,246382,12,HH-36903,Hush Hush,Released
8,Sep 14 2022 9:39AM,246382,12,HH-36904,Hush Hush,Released
9,Sep 14 2022 9:39AM,246382,12,HH-36905,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,246379,Released,1
38,246380,Released,7
39,246381,Released,1
40,246382,Released,15
41,246383,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
246379,NaN,1.0
246380,NaN,7.0
246381,NaN,1.0
246382,NaN,15.0
246383,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
246178,1.0,0.0
246182,0.0,1.0
246185,0.0,1.0
246195,1.0,0.0
246199,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
246178,1,0
246182,0,1
246185,0,1
246195,1,0
246199,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,246178,1,0
1,246182,0,1
2,246185,0,1
3,246195,1,0
4,246199,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,246178,1,
1,246182,,1
2,246185,,1
3,246195,1,
4,246199,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF
4,Sep 14 2022 9:39AM,246382,12,Hush Hush
19,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.
26,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC"
27,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample
28,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions
29,Sep 14 2022 9:02AM,246376,10,Bio-PRF
30,Sep 14 2022 8:45AM,246375,19,ACG North America LLC
33,Sep 14 2022 8:35AM,246374,16,Sartorius Bioprocess Solutions
34,Sep 14 2022 8:34AM,246373,16,Sartorius Stedim North America


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF,,4
1,Sep 14 2022 9:39AM,246382,12,Hush Hush,,15
2,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,,7
3,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",,1
4,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,,1
5,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions,,1
6,Sep 14 2022 9:02AM,246376,10,Bio-PRF,1,
7,Sep 14 2022 8:45AM,246375,19,ACG North America LLC,2,1
8,Sep 14 2022 8:35AM,246374,16,Sartorius Bioprocess Solutions,,1
9,Sep 14 2022 8:34AM,246373,16,Sartorius Stedim North America,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4,
1,Sep 14 2022 9:39AM,246382,12,Hush Hush,15,
2,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,7,
3,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",1,
4,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,1,
5,Sep 14 2022 9:25AM,246378,10,Acute Outpatient Solutions,1,
6,Sep 14 2022 9:02AM,246376,10,Bio-PRF,,1
7,Sep 14 2022 8:45AM,246375,19,ACG North America LLC,1,2
8,Sep 14 2022 8:35AM,246374,16,Sartorius Bioprocess Solutions,1,
9,Sep 14 2022 8:34AM,246373,16,Sartorius Stedim North America,,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4,
1,Sep 14 2022 9:39AM,246382,12,Hush Hush,15,
2,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,7,
3,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",1,
4,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4.0,NaN
1,Sep 14 2022 9:39AM,246382,12,Hush Hush,15.0,NaN
2,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,7.0,NaN
3,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",1.0,NaN
4,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Sep 14 2022 9:50AM,246383,10,Bio-PRF,4.0,0.0
1,Sep 14 2022 9:39AM,246382,12,Hush Hush,15.0,0.0
2,Sep 14 2022 9:38AM,246380,10,Eywa Pharma Inc.,7.0,0.0
3,Sep 14 2022 9:38AM,246381,19,"NAPP Technologies, LLC",1.0,0.0
4,Sep 14 2022 9:28AM,246379,17,TherapeuticsMD Sample,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2956077,32.0,1.0
102,985422,4.0,0.0
12,246382,15.0,0.0
16,492747,1.0,1.0
17,739102,3.0,0.0
19,739120,2.0,4.0
20,738965,3.0,0.0
21,3200771,11.0,2.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2956077,32.0,1.0
1,102,985422,4.0,0.0
2,12,246382,15.0,0.0
3,16,492747,1.0,1.0
4,17,739102,3.0,0.0
5,19,739120,2.0,4.0
6,20,738965,3.0,0.0
7,21,3200771,11.0,2.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,32.0,1.0
1,102,4.0,0.0
2,12,15.0,0.0
3,16,1.0,1.0
4,17,3.0,0.0
5,19,2.0,4.0
6,20,3.0,0.0
7,21,11.0,2.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,102,Released,4.0
2,12,Released,15.0
3,16,Released,1.0
4,17,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,17,19,20,21
Status,,,,,,,,
Executing,1.0,0.0,0.0,1.0,0.0,4.0,0.0,2.0
Released,32.0,4.0,15.0,1.0,3.0,2.0,3.0,11.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,17,19,20,21
0,Executing,1.0,0.0,0.0,1.0,0.0,4.0,0.0,2.0
1,Released,32.0,4.0,15.0,1.0,3.0,2.0,3.0,11.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,17,19,20,21
0,Executing,1.0,0.0,0.0,1.0,0.0,4.0,0.0,2.0
1,Released,32.0,4.0,15.0,1.0,3.0,2.0,3.0,11.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()